In [1]:
import numpy as np
import pandas as pd
import matplotlib as mt
import itertools as it
import Algorithm as al
import Support as sup

In [2]:
ds=pd.read_csv("C:/Users/Gulli/Desktop/Uni/Magi/MachineLearningandstat/Progetto_ML/your_dataset.csv")
X=ds.iloc[:,:-1]
Y=ds.iloc[:,-1]

In [ ]:
print(ds.isnull().sum())
ds = ds.dropna()

In [3]:
X, Y = sup.Outliers(X, Y)
X.reset_index(drop=True, inplace=True)
Y.reset_index(drop=True, inplace=True)

In [5]:
count1=np.sum(Y==1)
count_inv=np.sum(Y==-1)
print(f'{count1}, {count_inv},{Y.shape-count_inv}')

4571, 4885,[4571]


In [4]:
X_train, X_test, Y_train, Y_test = sup.Split(X, Y)
X_train, X_test = sup.Standardized(X_train, X_test)

TUNING

In [5]:
perceptron_param = {
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

pegasos_param = {
    'lambda_par': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

logistic_param = {
    'lambda_par': [0.001, 0.01, 0.1, 1],
    'n': [50, 100, 200, 500],
}

In [19]:
loss_perc_in, param_perc_in = sup.tuning_par(al.Perceptron, pd.DataFrame(X_train), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos_in, param_pegasos_in = sup.tuning_par(al.PegasosSVM, pd.DataFrame(X_train), pd.Series(Y_train), pegasos_param, 5)
loss_logit_in, param_logit_in = sup.tuning_par(al.RegLogisticClass, pd.DataFrame(X_train), pd.Series(Y_train), logistic_param, 5)
print(param_perc_in, loss_perc_in)
print(param_pegasos_in, loss_pegasos_in)
print(param_logit_in, loss_logit_in)

{'learning_rate': 0.001, 'n': 50} 0.31679217781058666
{'lambda_par': 0.001, 'n': 500} 0.33281135279197505
{'lambda_par': 1, 'n': 200} 0.5030851191686588


In [ ]:
correlation_matrix = pd.DataFrame(X_train).corr()
high_corr = correlation_matrix[(correlation_matrix > 0.9) | (correlation_matrix < -0.9)]
high_corr = high_corr[high_corr != 1.0]
high_corr.stack().reset_index()

In [43]:
perceptron_model=al.Perceptron(0.1, 200)
pegasos_model=al.PegasosSVM(0.001, 500)
logistic_model=al.RegLogisticClass(0.01, 500)
perceptron_model.fit(X_train, Y_train)
pegasos_model.fit(X_train, Y_train)
logistic_model.fit(X_train, Y_train)

In [44]:
loss_perceptron = sup.Zero_One_Loss(Y_test, perceptron_model.predict(X_test))
loss_pegasos = sup.Zero_One_Loss(Y_test, pegasos_model.predict(X_test))
loss_logistic = sup.Zero_One_Loss(Y_test, logistic_model.predict(X_test))
print(f"0-1 Loss Perceptron : {loss_perceptron}  ")
print(f"0-1 Loss Pegasos : {loss_pegasos}  ")
print(f"0-1 Loss Logistic : {loss_logistic}  ")

0-1 Loss Perceptron : 0.3316884032428622  
0-1 Loss Pegasos : 0.31300669721536833  
0-1 Loss Logistic : 0.5597462107860416  


In [28]:
#puoi scrivere funzione
np.random.seed(42)
models=['Perceptron', 'Pegasos', 'Logistic']
for m in models:
    for i in [2,5,9,[5,9],[2,5],[2,9],[2,5,9]]:
        X_reduced_train=np.delete(X_train, i, axis=1)
        X_reduced_test=np.delete(X_test, i, axis=1)
        if m == 'Perceptron':
            model = al.Perceptron(0.1,200)
        elif m == 'Pegasos':
            model = al.PegasosSVM(0.001,500)
        else:
            model = al.RegLogisticClass(0.01,500)
        
        model.fit(X_reduced_train, Y_train)
        y_pred_tr = model.predict(X_reduced_train)
        y_pred_test = model.predict(X_reduced_test)
   
        loss_perceptron_tr = sup.Zero_One_Loss(Y_train, y_pred_tr)
        loss_perceptron_test = sup.Zero_One_Loss(Y_test, y_pred_test)
        print(f"0-1 Loss Perceptron (Training): {loss_perceptron_tr}  ")
        print(f"0-1 Loss Perceptron (Test): {loss_perceptron_test}  ") 

0-1 Loss Perceptron (Training): 0.3361534975071763  
0-1 Loss Perceptron (Test): 0.35671483961931616  
0-1 Loss Perceptron (Training): 0.3449161504759027  
0-1 Loss Perceptron (Test): 0.35988720479379627  
0-1 Loss Perceptron (Training): 0.3370599788487687  
0-1 Loss Perceptron (Test): 0.35107507930912935  
0-1 Loss Perceptron (Training): 0.34823991539507476  
0-1 Loss Perceptron (Test): 0.3683468452590765  
0-1 Loss Perceptron (Training): 0.35881553104698594  
0-1 Loss Perceptron (Test): 0.37151921043355657  
0-1 Loss Perceptron (Training): 0.3038223296570479  
0-1 Loss Perceptron (Test): 0.3228762777581953  
0-1 Loss Perceptron (Training): 0.6286448103943194  
0-1 Loss Perceptron (Test): 0.6087416284807896  
0-1 Loss Perceptron (Training): 0.32195195648889563  
0-1 Loss Perceptron (Test): 0.35001762425096933  
0-1 Loss Perceptron (Training): 0.4768091856775948  
0-1 Loss Perceptron (Test): 0.4797321113852661  


In [5]:
#To simplify our analysis we're gonna remove the same feature for all the models
X_train=np.delete(X_train, 9, axis=1)
X_test=np.delete(X_test, 9, axis=1) 

DOING TUNING AGAIN WITHOUT THE LAST FEATURE AND SEEING IF THE BEST PARAMETERS CHANGE

In [ ]:
loss_perc, param_perc = sup.tuning_par(al.Perceptron, pd.DataFrame(X_train), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos, param_pegasos = sup.tuning_par(al.PegasosSVM, pd.DataFrame(X_train), pd.Series(Y_train), pegasos_param, 5)
loss_logit, param_logit = sup.tuning_par(al.RegLogisticClass, pd.DataFrame(X_train), pd.Series(Y_train), logistic_param, 5)
print(param_perc, loss_perc)
print(param_pegasos, loss_pegasos)
print(param_logit, loss_logit)

In [6]:
perceptron_model=al.Perceptron(0.1, 200)
pegasos_model=al.PegasosSVM(0.001, 500)
logistic_model=al.RegLogisticClass(0.01, 500)
perceptron_model.fit(X_train, Y_train)
pegasos_model.fit(X_train, Y_train)
logistic_model.fit(X_train, Y_train)
loss_perceptron = sup.Zero_One_Loss(Y_test, perceptron_model.predict(X_test))
loss_pegasos = sup.Zero_One_Loss(Y_test, pegasos_model.predict(X_test))
loss_logistic = sup.Zero_One_Loss(Y_test, logistic_model.predict(X_test))
print(f"0-1 Loss Perceptron : {loss_perceptron}  ")
print(f"0-1 Loss Pegasos : {loss_pegasos}  ")
print(f"0-1 Loss Logistic : {loss_logistic}  ")


0-1 Loss Perceptron : 0.3316884032428622  
0-1 Loss Pegasos : 0.3260486429326754  
0-1 Loss Logistic : 0.48501938667606626  


In [7]:
col_names = [f'x{i+1}' for i in range(X.shape[1])]
col_names=np.delete(col_names, 5)
sup.weights_comparison(perceptron_model.weights, pegasos_model.weights, logistic_model.weights, col_names)

NameError: name 'perceptron_model' is not defined

POLYNOMIAL EXPANSION

In [8]:
X_poly_train, X_poly_test = sup.Polynomial_exp(X_train,X_test, col_names)
X_poly_train, X_poly_test = sup.Standardized(X_poly_train, X_poly_test)

NameError: name 'col_names' is not defined

In [ ]:
correlation_matrix_poly = X_poly_train.corr()
high_corr_poly = correlation_matrix_poly[(correlation_matrix_poly > 0.9) | (correlation_matrix_poly < -0.9)]
high_corr_poly = high_corr_poly[high_corr_poly != 1.0]
high_corr_poly.stack().reset_index()

In [10]:
loss_perc_poly, param_perc_poly = sup.tuning_par(al.Perceptron, pd.DataFrame(X_poly_train), pd.Series(Y_train), perceptron_param ,5)
loss_pegasos_poly, param_pegasos_poly = sup.tuning_par(al.PegasosSVM, pd.DataFrame(X_poly_train), pd.Series(Y_train), pegasos_param, 5)
loss_logit_poly, param_logit_poly = sup.tuning_par(al.RegLogisticClass, pd.DataFrame(X_poly_train), pd.Series(Y_train), logistic_param, 5)
print(param_perc_poly, loss_perc_poly)
print(param_pegasos_poly, loss_pegasos_poly)
print(param_logit_poly, loss_logistic_poly)

In [10]:
perceptron_model_poly = al.Perceptron(1, 200)
pegasos_model_poly = al.PegasosSVM(0.1, 500)
logistic_model_poly = al.RegLogisticClass(0.01, 100)
perceptron_model_poly.fit(X_poly_train, Y_train)
pegasos_model_poly.fit(X_poly_train, Y_train)
logistic_model_poly.fit(X_poly_train, Y_train)

In [16]:
loss_perceptron_poly = sup.Zero_One_Loss(Y_test, perceptron_model_poly.predict(X_poly_test))
loss_pegasos_poly = sup.Zero_One_Loss(Y_test, pegasos_model_poly.predict(X_poly_test))
loss_logistic_poly = sup.Zero_One_Loss(Y_test, logistic_model_poly.predict(X_poly_test))
print(f"0-1 Loss Perceptron : {loss_perceptron_poly}")
print(f"0-1 Loss Pegasos : {loss_pegasos_poly}")
print(f"0-1 Loss Logistic : {loss_logistic_poly}")

0-1 Loss Perceptron : 0.10715544589354953
0-1 Loss Pegasos : 0.33944307366936904
0-1 Loss Logistic : 0.5830102220655622


In [ ]:
sup.weights_comparison(perceptron_model_poly.weights, pegasos_model_poly.weights, logistic_model_poly.weights, X_poly_train.columns)

KERNEL

In [6]:
kernel_perc = al.KernelizedPerceptron('gaussian')
kernel_perc.fit(X_train, Y_train)
y_pred, weights = kernel_perc.predict(X_test)

KeyboardInterrupt: 

In [ ]:
kernel_perc_pol = al.KernelizedPerceptron('polynomial')
kernel_perc_pol.fit(X_train, Y_train)
y_pred_pol, weights_pol = kernel_perc_pol.predict(X_test)

In [ ]:
kernel_pegasos_gaussian = al.KernelizedPegasos('gaussian')
kernel_pegasos_gaussian.fit(X_train, Y_train)
y_pred_pegasos, weights_pegasos = kernel_pegasos_gaussian.predict(X_test)